In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando os dados
treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

treino.shape, teste.shape

In [ ]:
# Unindo os dois dataframes
df_all = treino.append(teste)

df_all.shape

In [ ]:
# Visualizando os dois dataframe# Análise das características dos dados
df_all.info()
df_all

In [ ]:
# Analisando os tipos de dados
df_all.info()

In [ ]:
# Analise estatística dos dados númericos
df_all.describe().T

In [ ]:
# Verificando o tipo de dados object
df_all.select_dtypes('object').head()

In [ ]:
# Observando o cabeçalho dos 10 primeiros registros, depois das modificações
df_all.head(10)

In [ ]:
# Verificando valores únicos nas colunas do tipo object
df_all[['Gender','Customer Type','Type of Travel','Class','satisfaction']].nunique()

In [ ]:
# Olhando a variável target
df_all['satisfaction'].value_counts()

In [ ]:
# Transformando as variáveis em binarias - neutral or dissatisfied = 0, satisfied = 1.
df_all['satisfaction'] = df_all['satisfaction'].replace({'satisfied': 1, 'neutral or dissatisfied' : 0}).astype(int)

In [ ]:
# Analisando a coluna satisfaction depois da trasnformação
df_all['satisfaction'].value_counts()

In [ ]:
# Analisando a coluna Gender
df_all['Gender'].value_counts()

In [ ]:
#Transformação dos dados Female = 0, Male = 1.
df_all['Gender'] = df_all['Gender'].replace({'Male': 1, 'Female' : 0}).astype(int)

In [ ]:
# Analisando a coluna Gender depois da transformação
df_all['Gender'].value_counts()

In [ ]:
# Analisando a coluna Customer Type
df_all['Customer Type'].value_counts()

In [ ]:
# Transformação dos dados - Loyal = 0, disloyal = 1.
df_all['Customer Type'] = df_all['Customer Type'].replace({'Loyal Customer': 0, 'disloyal Customer' : 1}).astype(int)

In [ ]:
# Analisando a coluna Customer Type depois das transformações
df_all['Customer Type'].value_counts()

In [ ]:
# Analisando a coluna Type of Travel
df_all['Type of Travel'].value_counts()

In [ ]:
# Transformação dos dados - business = 0, personal = 1.
df_all['Type of Travel'] = df_all['Type of Travel'].replace({'Business travel': 0, 'Personal Travel' : 1}).astype(int)

In [ ]:
# Analisando a coluna Type of Travel depois das transformações
df_all['Type of Travel'].value_counts()

In [ ]:
# Analisando a coluna Class
df_all['Class'].value_counts()

In [ ]:
# Transformação dos Dados - business = 0, eco = 1; eco plus = 2.
df_all['Class'] = df_all['Class'].replace({'Business': 0, 'Eco' : 1, 'Eco Plus' : 2}).astype(int)

In [ ]:
# Analisando a coluna Class depois das transformações
df_all['Class'].value_counts()

In [ ]:
# Visualização dos Dados após as trasnformações
df_all.head()

In [ ]:
# campos object convertidos para int
df_all.info()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

In [ ]:
df_all['Arrival Delay in Minutes'].median()

In [ ]:
# substituindo o valor pela mediana
df_all[df_all['Arrival Delay in Minutes'].isnull()] = df_all['Arrival Delay in Minutes'].median()

In [ ]:
# Verificando os valores nulos após as transformações
df_all.isnull().sum()

# Modelo de Regressão - aqui e aonde o filho chora e a mãe não ver

In [ ]:
# Instanciando o modelo random forest classifier
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=420)
rfc2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=420)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['satisfaction']]
feats

In [ ]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando as bases de treino e teste
treino, teste = train_test_split(df_all, test_size=0.20, random_state=42)

# Separando as bases de treino e validação
treino, validacao = train_test_split(treino, test_size=0.20, random_state=42)

treino.shape, validacao.shape, teste.shape

In [ ]:
# Vamos treinar o modelo - Treino
rfc.fit(treino[feats], treino['satisfaction'])

In [ ]:
# Vamos treinar o modelo - Validacao
rfc2.fit(validacao[feats], validacao['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_validacao = rfc2.predict(validacao[feats])

preds_validacao

In [ ]:
# Analisando o desempenho do modelo

# Vamos as métricas
from sklearn.metrics import accuracy_score

In [ ]:
# Acurácia das previsões de validação
accuracy_score(validacao['satisfaction'], preds_validacao)

Até estranhei porque a acurácia está alta, porém verifiquei nos outros nootbooks estão na mesma ideia.

In [ ]:
# Medindo a acurácia nos dados de teste
preds_teste = rfc.predict(teste[feats])

accuracy_score(teste['satisfaction'], preds_teste)

Até estranhei porque a acurácia está alta, porém verifiquei nos outros nootbooks estão na mesma ideia.

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada) - TESTE
pd.Series(rfc.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada) -  Validação
pd.Series(rfc2.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(validacao['satisfaction'], preds_validacao)

In [ ]:
# Matriz de Confusão - Dados de TESTE
skplt.metrics.plot_confusion_matrix(teste['satisfaction'], preds_teste)